In [1]:
import os
os.environ['SPOTIPY_CLIENT_ID'] = '0dd677ab735f4fd1b9dbf6b236350ba1'
os.environ['SPOTIPY_CLIENT_SECRET'] = 'bbe8736a14ba4e64bfb2d4103c8957aa'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://google.com/'

In [2]:
from OOSpotify import *

In [7]:
a = Artist('kanye')

In [17]:
b = a.getAlbums()
[i.name for i in b]

['KIDS SEE GHOSTS',
 'Yeezus',
 'ye',
 'The Life Of Pablo',
 'Graduation',
 'The College Dropout',
 'Late Registration',
 'My Beautiful Dark Twisted Fantasy',
 'The College Dropout (Edited)',
 '808s & Heartbreak',
 'My Beautiful Dark Twisted Fantasy',
 'Late Registration',
 'Graduation']

In [32]:
print(b[4].uri)
print(b[4].Tracks())
print(b[12].uri)
print(b[12].Tracks())

spotify:album:3SZr5Pco2oqKFORCP3WNj9
0: Good Morning
1: Champion
2: Stronger
3: I Wonder
4: Good Life
5: Can't Tell Me Nothing
6: Barry Bonds
7: Drunk and Hot Girls
8: Flashing Lights
9: Everything I Am
10: The Glory
11: Homecoming
12: Big Brother
None
spotify:album:7nxBMFwJtTTKDNxVCKmYi3
0: Good Morning
1: Champion
2: Stronger
3: I Wonder
4: Good Life
5: Can't Tell Me Nothing
6: Barry Bonds
7: Drunk and Hot Girls
8: Flashing Lights
9: Everything I Am
10: The Glory
11: Homecoming
12: Big Brother
None


In [ ]:
%%timeit 
a = Artist('Michael Jackson').Albums()

In [ ]:
################################################################
# Simple Functions
def getPlaylists(userID):
    playlists = sp.user_playlists(userID)['items']
    return playlists
#def findPlaylistID(userID,playlistName):
#    playlists = getPlaylists(userID)
#    return [playlist['id'] for playlist in playlists if playlist['name'] == playlistName][0]

def createPlaylist(userID,playlistName):
    playlists = getPlaylists(userID)
    for playlist in playlists:
        if playlist['name'] == playlistName:
            ans = input('Playlist with this name already exists. Do you want to overwrite it?')
            if ans == 'y':
                sp.user_playlist_replace_tracks(userID,playlist['id'],tracks=[])
            else:
                print('Exiting...')
                sys.exit()
            return playlist['id']
    sp.user_playlist_create(userID,playlistName)  

def addTracksToPlaylist(userID,playlistID,trackIDs):
    sp.user_playlist_add_tracks(userID,playlistID,trackIDs)

#Creates a dictionary of dictionaries for each artist
def getArtists(artists):
    artistDict = {}
    for artist in artists:
        a = Artist(artist)
        artistDict[a.name.replace(' ','_')] = a
    return artistDict          
#################################################################            
# Less Simple Funcions
    
def createPlaylistFromTopTracks(artists,userID,playlistName=None,reqs=None):
    if not playlistName:
        playlistName = ' + '.join(artists)
    playlistID = createPlaylist(userID,playlistName)
    
    trackIDs = []
    for artist in artists:
        a = Artist(artist)
        trackIDs += [i.id for i in a.getTopTracks()]
        print('Top Tracks for {}:'.format(a.name))
        a.TopTracks()
        print()
    shuffle(trackIDs)

    addTracksToPlaylist(userID,playlistID,trackIDs)

def satisfyReqs(trackObj,reqs=None):
    if reqs:
        perf = []
        for key,val in reqs.items():
            perf.append(eval('{}{}{}'.format(getattr(trackObj,key),val[0],val[1])))
        return all(perf)
    else:
        return True

def createRadioPlaylist(artists,userID,playlistName=None,reqs=None):
    if not playlistName:
        playlistName= ' x '.join(artists)+' Radio'
    playlistID = createPlaylist(userID,playlistName)
    
    trackIDs = []
    reccArtists = []
    for artist in artists:
        a = Artist(artist)
        reccArtists += a.getRelatedArtists()[0:5] # Big toggle here
        tracks = [i for i in a.getTopTracks() if satisfyReqs(i,reqs)]
        #print([(i.name,i.tempo,i.artists[0]['name']) for i in tracks])
        trackIDs += [i.id for i in tracks]
    shuffle(reccArtists)
    #print([i.name for i in reccArtists])
    for a in reccArtists:
        if len(trackIDs)<30:
            tracks = [i for i in a.getTopTracks() if satisfyReqs(i,reqs)]
            #print([(i.name,i.tempo,i.artists[0]['name']) for i in tracks])
            trackIDs += [i.id for i in tracks]
        else:
            break
    #print(len(trackIDs))
    shuffle(trackIDs)
    
    addTracksToPlaylist(userID,playlistID,trackIDs)
        
def createPlaylistFromAlbumSpecs(artists):
    pass

def createPlaylistFromProducers(producers):
    pass
    #Suggestion from Katie -> need to find how to get this info
    
    #add in restriction for 'albums
    

In [ ]:
a = ['d','e','f']
print('\n'.join('{}'.format(i) for i in a))